In [1]:

import sys
!pip install tensorflow>=2.4.0

import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.18.0


In [2]:
# Cell 2: Mount Google Drive
# This cell connects your notebook to Google Drive to access your model file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
MODEL_PATH = '/content/drive/MyDrive/Potato model/Model.h5'
print(f"Model path set to: {MODEL_PATH}")


Model path set to: /content/drive/MyDrive/Potato model/Model.h5


In [5]:
# Cell 4: Load the Model from Google Drive
# This cell loads your model and displays its architecture
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print("✅ Model loaded successfully from Google Drive!")

    # Display model summary to understand what we're working with
    model.summary()
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please check that the path to your model is correct.")


✅ Model loaded successfully from Google Drive!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 64)        │           4,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       6,423,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,540,229 (24.95 MB)

 Trainable params: 6,540,227 (24.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [6]:
# This cell converts your model to the basic TensorFlow Lite format
try:
    # Basic TFLite conversion
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the basic TFLite model to Drive
    tflite_path = MODEL_PATH.replace('.h5', '.tflite')
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)
    print(f"✅ Basic TFLite model saved to {tflite_path}")
    print(f"Size: {len(tflite_model) / 1024:.2f} KB")
except Exception as e:
    print(f"❌ Error during basic conversion: {e}")


Saved artifact at '/tmp/tmppebheskd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  137811222234128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222236816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222238736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222236240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222242192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222242960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222242768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222243152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222243344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222244496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222241616

In [7]:
# This cell applies basic quantization to further reduce model size
try:
    # Quantized model conversion with default optimizations
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    quantized_tflite_model = converter.convert()

    # Save the quantized TFLite model to Drive
    quantized_path = MODEL_PATH.replace('.h5', '_quantized.tflite')
    with open(quantized_path, 'wb') as f:
        f.write(quantized_tflite_model)
    print(f"✅ Quantized TFLite model saved to {quantized_path}")
    print(f"Size: {len(quantized_tflite_model) / 1024:.2f} KB")
except Exception as e:
    print(f"❌ Error during quantization: {e}")

Saved artifact at '/tmp/tmpmnkpvuhm'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  137811222234128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222236816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222238736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222236240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222242192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222242960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222242768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222243152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222243344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222244496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137811222241616

In [9]:
# This cell compares the sizes of all model versions
try:
    import os
    original_size = os.path.getsize(MODEL_PATH) / 1024

    print(" Model Size Comparison:")
    print(f"Original H5 model:     {original_size:.2f} KB")
    print(f"Basic TFLite model:    {len(tflite_model) / 1024:.2f} KB")
    print(f"Quantized TFLite model: {len(quantized_tflite_model) / 1024:.2f} KB")

    print(f"\nSize reduction from basic to quantized: {(1 - len(quantized_tflite_model) / len(tflite_model)) * 100:.2f}%")
    print(f"Total size reduction from original: {(1 - len(quantized_tflite_model) / original_size) * 100:.2f}%")
except Exception as e:
    print(f"❌ Error comparing sizes: {e}")

 Model Size Comparison:
Original H5 model:     76700.55 KB
Basic TFLite model:    25552.25 KB
Quantized TFLite model: 6402.84 KB

Size reduction from basic to quantized: 74.94%
Total size reduction from original: -8448.19%


In [ ]:

# This cell attempts full integer quantization for maximum optimization
# Note: You need to provide real data samples for best results

def representative_dataset_gen():
    """
    This function provides sample data to the quantizer.
    Replace the random data with actual samples from your dataset.
    """
    # Get the input shape from your model
    input_shape = model.input_shape
    # Remove batch dimension if it's None
    if input_shape[0] is None:
        input_shape = (1,) + input_shape[1:]

    # Generate random data - REPLACE THIS with real data samples
    for _ in range(100):  # Generate 100 sample inputs
        yield [tf.random.uniform(input_shape)]

try:
    print("\n🔬 Attempting Full Integer Quantization...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    full_int_model = converter.convert()

    full_int_path = MODEL_PATH.replace('.h5', '_full_integer.tflite')
    with open(full_int_path, 'wb') as f:
        f.write(full_int_model)
    print(f"✅ Full integer model saved to {full_int_path}")
    print(f"Size: {len(full_int_model) / 1024:.2f} KB")

    print("\n📊 Final Model Size Comparison:")
    print(f"Original H5 model:      {original_size:.2f} KB")
    print(f"Basic TFLite model:     {len(tflite_model) / 1024:.2f} KB")
    print(f"Quantized TFLite model: {len(quantized_tflite_model) / 1024:.2f} KB")
    print(f"Full Integer model:     {len(full_int_model) / 1024:.2f} KB")
except Exception as e:
    print(f"❌ Note: Full integer quantization failed with error: {e}")
    print("This usually means you need to provide real representative data for your model inputs.")


In [10]:
# This cell lets you download models directly to your computer from Colab
from google.colab import files

print("\n📥 Download models directly to your computer:")
print("Run any of these commands to download specific model versions:")
print("files.download('{}')  # Download basic TFLite model".format(tflite_path))
print("files.download('{}')  # Download quantized TFLite model".format(quantized_path))
try:
    print("files.download('{}')  # Download full integer TFLite model".format(full_int_path))
except:
    pass


📥 Download models directly to your computer:
Run any of these commands to download specific model versions:
files.download('/content/drive/MyDrive/Potato model/Model.tflite')  # Download basic TFLite model
files.download('/content/drive/MyDrive/Potato model/Model_quantized.tflite')  # Download quantized TFLite model
